In [1]:
import pandas as pd
import numpy as np
import ta
import talib
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import datetime

import util_functions as uf

In [32]:
sma_value = [50, 100, 150, 200]
close='Close'
sym='ADBE'

today = datetime.datetime.now()
end_date = datetime.datetime(today.year, today.month, today.day-1)
start_date = datetime.datetime(today.year-1, 1, 1)

In [112]:
df = uf.getdata(sym, sma_value, close, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [113]:
df['rsi'] = ta.momentum.rsi(df[close], window=50)
df.dropna(subset=['sma_200','rsi'],inplace=True)

In [114]:
# Calculate MACD
df['exp12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['exp26'] = df['Close'].ewm(span=26, adjust=False).mean()
df['macd'] = df['exp12'] - df['exp26']
df['signal'] = df['macd'].ewm(span=9, adjust=False).mean()
df['histogram'] = df['macd'] - df['signal']

In [115]:
df

,Date,Open,High,Low,Close,Volume,ticker,sma_50,sma_100,sma_150,sma_200,rsi,exp12,exp26,macd,signal,histogram
199,2023-10-18,554.479980,568.179993,554.239990,557.869995,2614500,ADBE,531.985201,510.912699,463.024266,435.211499,57.992242,557.869995,557.869995,0.000000,0.000000,0.000000
200,2023-10-19,560.000000,567.099976,553.469971,555.739990,2809300,ADBE,532.824401,512.316199,464.373932,436.305599,57.640619,557.542302,557.712217,-0.169915,-0.033983,-0.135932
201,2023-10-20,555.739990,555.979980,535.950012,540.960022,2728100,ADBE,533.327001,513.553699,465.592732,437.303349,55.267895,554.991182,556.471314,-1.480132,-0.323213,-1.156919
202,2023-10-23,535.809998,548.030029,531.000000,540.409973,2172600,ADBE,533.966200,514.779899,466.776265,438.363199,55.181634,552.747919,555.281585,-2.533666,-0.765303,-1.768362
203,2023-10-24,544.640015,546.000000,532.119995,539.559998,2358100,ADBE,534.312400,515.907999,467.878532,439.397249,55.046162,550.719008,554.117023,-3.398015,-1.291846,-2.106169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,2024-03-22,509.070007,511.589996,496.670013,499.519989,5410300,ADBE,577.990599,585.775000,568.612000,552.042000,43.041568,529.735528,548.129806,-18.394278,-14.377426,-4.016852
307,2024-03-25,496.790009,510.630005,496.690002,507.230011,4277000,ADBE,576.185400,585.577900,568.606000,552.486550,43.939674,526.273141,545.100191,-18.827051,-15.267351,-3.559700
308,2024-03-26,508.059998,512.789978,503.589996,507.600006,2813300,ADBE,574.406600,585.333300,568.520200,552.829400,43.982926,523.400351,542.322400,-18.922049,-15.998291,-2.923758
309,2024-03-27,515.469971,516.859985,499.700012,504.399994,4228400,ADBE,572.541000,584.932300,568.419667,553.081400,43.685475,520.477219,539.513333,-19.036114,-16.605855,-2.430259


In [116]:
def generate_signal(x):
    if (min(x['Open'], x['Close']) > x['sma_200']) & (x['macd']>=x['signal']) & (x['macd']<0):
        return 1
    elif (min(x['Open'], x['Close']) > x['sma_200']) & (x['macd']<=x['signal']) & (x['macd']>0): 
        return -1
    elif (min(x['Open'], x['Close']) < x['sma_200']):
        return -1
    else:
        return 0
    

In [117]:
df['trade_signal'] = df.apply(lambda row: generate_signal(row), axis=1)

In [118]:
df_trade = uf.get_df_trade_position(df, 'position','trade_signal')

c:\Users\natal\OneDrive\Documents\Python Scripts\Quantitative_Projects\Stock_Analysis\Mean_Reverion_with_rsi\util_functions.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\natal\OneDrive\Documents\Python Scripts\Quantitative_Projects\Stock_Analysis\Mean_Reverion_with_rsi\util_functions.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [119]:
df_trade

,Date,Open,High,Low,Close,Volume,ticker,sma_50,sma_100,sma_150,sma_200,rsi,exp12,exp26,macd,signal,histogram,trade_signal,position
210,2023-11-02,550.440002,559.359985,550.000000,558.710022,2634800,ADBE,535.954401,522.071299,475.161666,445.934899,56.771616,539.405827,544.746750,-5.340924,-6.340366,0.999443,1,1
230,2023-12-01,609.239990,615.200012,603.179993,612.469971,2490000,ADBE,558.493599,544.810099,504.991332,468.901799,60.410098,609.113708,593.649009,15.464699,15.603018,-0.138319,-1,-1
258,2024-01-12,598.390015,599.380005,593.020020,596.539978,1477000,ADBE,596.260000,565.577000,545.636999,503.725499,54.711331,588.864870,591.484095,-2.619226,-3.392279,0.773053,1,1
276,2024-02-08,615.849976,621.340027,612.510010,615.859985,1815700,ADBE,604.183600,577.934400,561.464799,524.876599,54.928655,617.071104,610.450581,6.620523,6.966168,-0.345644,-1,-1
296,2024-03-08,553.000000,560.440002,550.510010,551.690002,2831100,ADBE,587.760800,586.254301,567.554667,545.881200,47.269734,558.004539,570.562616,-12.558076,-12.652198,0.094122,1,1
301,2024-03-15,499.640015,503.130005,485.000000,492.459991,19570500,ADBE,584.479999,586.360500,568.682667,549.957100,41.698930,553.413623,564.911370,-11.497747,-10.226877,-1.270870,-1,-1


In [120]:
df = df.merge(df_trade[['Date','position']], how='left', on='Date')

In [121]:
df

,Date,Open,High,Low,Close,Volume,ticker,sma_50,sma_100,sma_150,sma_200,rsi,exp12,exp26,macd,signal,histogram,trade_signal,position
0,2023-10-18,554.479980,568.179993,554.239990,557.869995,2614500,ADBE,531.985201,510.912699,463.024266,435.211499,57.992242,557.869995,557.869995,0.000000,0.000000,0.000000,0,NaN
1,2023-10-19,560.000000,567.099976,553.469971,555.739990,2809300,ADBE,532.824401,512.316199,464.373932,436.305599,57.640619,557.542302,557.712217,-0.169915,-0.033983,-0.135932,0,NaN
2,2023-10-20,555.739990,555.979980,535.950012,540.960022,2728100,ADBE,533.327001,513.553699,465.592732,437.303349,55.267895,554.991182,556.471314,-1.480132,-0.323213,-1.156919,0,NaN
3,2023-10-23,535.809998,548.030029,531.000000,540.409973,2172600,ADBE,533.966200,514.779899,466.776265,438.363199,55.181634,552.747919,555.281585,-2.533666,-0.765303,-1.768362,0,NaN
4,2023-10-24,544.640015,546.000000,532.119995,539.559998,2358100,ADBE,534.312400,515.907999,467.878532,439.397249,55.046162,550.719008,554.117023,-3.398015,-1.291846,-2.106169,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2024-03-22,509.070007,511.589996,496.670013,499.519989,5410300,ADBE,577.990599,585.775000,568.612000,552.042000,43.041568,529.735528,548.129806,-18.394278,-14.377426,-4.016852,-1,NaN
108,2024-03-25,496.790009,510.630005,496.690002,507.230011,4277000,ADBE,576.185400,585.577900,568.606000,552.486550,43.939674,526.273141,545.100191,-18.827051,-15.267351,-3.559700,-1,NaN
109,2024-03-26,508.059998,512.789978,503.589996,507.600006,2813300,ADBE,574.406600,585.333300,568.520200,552.829400,43.982926,523.400351,542.322400,-18.922049,-15.998291,-2.923758,-1,NaN
110,2024-03-27,515.469971,516.859985,499.700012,504.399994,4228400,ADBE,572.541000,584.932300,568.419667,553.081400,43.685475,520.477219,539.513333,-19.036114,-16.605855,-2.430259,-1,NaN


In [122]:
def pointpos_pattern(x):
    if x['position']==-1:
        return x['High']+3
    elif x['position']==1:
        return x['Low']-3
    else:
        return np.nan
df['pointpos_pattern'] = df.apply(lambda row: pointpos_pattern(row), axis=1)

In [123]:
df.position.value_counts()

position
 1.0    3
-1.0    3
Name: count, dtype: int64

In [124]:
# Calculate Bollinger Bands
indicator_bb = ta.volatility.BollingerBands(close=df[close], window=20, window_dev=2)

# Get Bollinger Bands
df['bb_upper'] = indicator_bb.bollinger_hband()
df['bb_lower'] = indicator_bb.bollinger_lband()

In [125]:
df

,Date,Open,High,Low,Close,Volume,ticker,sma_50,sma_100,sma_150,sma_200,rsi,exp12,exp26,macd,signal,histogram,trade_signal,position,pointpos_pattern,bb_upper,bb_lower
0,2023-10-18,554.479980,568.179993,554.239990,557.869995,2614500,ADBE,531.985201,510.912699,463.024266,435.211499,57.992242,557.869995,557.869995,0.000000,0.000000,0.000000,0,NaN,NaN,NaN,NaN
1,2023-10-19,560.000000,567.099976,553.469971,555.739990,2809300,ADBE,532.824401,512.316199,464.373932,436.305599,57.640619,557.542302,557.712217,-0.169915,-0.033983,-0.135932,0,NaN,NaN,NaN,NaN
2,2023-10-20,555.739990,555.979980,535.950012,540.960022,2728100,ADBE,533.327001,513.553699,465.592732,437.303349,55.267895,554.991182,556.471314,-1.480132,-0.323213,-1.156919,0,NaN,NaN,NaN,NaN
3,2023-10-23,535.809998,548.030029,531.000000,540.409973,2172600,ADBE,533.966200,514.779899,466.776265,438.363199,55.181634,552.747919,555.281585,-2.533666,-0.765303,-1.768362,0,NaN,NaN,NaN,NaN
4,2023-10-24,544.640015,546.000000,532.119995,539.559998,2358100,ADBE,534.312400,515.907999,467.878532,439.397249,55.046162,550.719008,554.117023,-3.398015,-1.291846,-2.106169,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2024-03-22,509.070007,511.589996,496.670013,499.519989,5410300,ADBE,577.990599,585.775000,568.612000,552.042000,43.041568,529.735528,548.129806,-18.394278,-14.377426,-4.016852,-1,NaN,NaN,595.987900,494.070102
108,2024-03-25,496.790009,510.630005,496.690002,507.230011,4277000,ADBE,576.185400,585.577900,568.606000,552.486550,43.939674,526.273141,545.100191,-18.827051,-15.267351,-3.559700,-1,NaN,NaN,595.342534,489.390470
109,2024-03-26,508.059998,512.789978,503.589996,507.600006,2813300,ADBE,574.406600,585.333300,568.520200,552.829400,43.982926,523.400351,542.322400,-18.922049,-15.998291,-2.923758,-1,NaN,NaN,594.963171,485.280835
110,2024-03-27,515.469971,516.859985,499.700012,504.399994,4228400,ADBE,572.541000,584.932300,568.419667,553.081400,43.685475,520.477219,539.513333,-19.036114,-16.605855,-2.430259,-1,NaN,NaN,594.433547,481.068458


In [126]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.8, 0.2])

candlestick =   go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])

colors = ['green' if val > 0 else 'red' for val in df['histogram']]

sma_200 = go.Scatter(x=df.index, y=df['sma_200'], mode='lines', name='SMA200', line=dict(color="orange"))
macd = go.Scatter(x=df.index, y=df['macd'], mode='lines', name='MACD', line=dict(color="blue"))
signal = go.Scatter(x=df.index, y=df['signal'], mode='lines', name='signal', line=dict(color="orange"))
histogram = go.Bar(x=df.index, y=df['histogram'], name='Histogram', marker=dict(color=colors))
signal_sell =  go.Scatter(x=df[df['position']==1].index, y=df[df['position']==1]['pointpos_pattern'], mode='markers', name='signal', marker=dict(color="orange",symbol='arrow-bar-up'))
signal_buy =  go.Scatter(x=df[df['position']==-1].index, y=df[df['position']==-1]['pointpos_pattern'], mode='markers', name='signal', marker=dict(color="orange",symbol='arrow-bar-down'))

fig.add_trace(candlestick, row=1, col=1)
fig.add_trace(sma_200, row=1, col=1)
fig.add_trace(macd, row=2, col=1)
fig.add_trace(signal, row=2, col=1)
fig.add_trace(histogram, row=2, col=1)
# fig.add_trace(sma200, row=1, col=1)
# fig.add_trace(bb_upper, row=1, col=1)
# fig.add_trace(bb_lower, row=1, col=1)
fig.add_trace(signal_sell, row=1, col=1)
fig.add_trace(signal_buy, row=1, col=1)

# rsi = go.Scatter(x=df.index, y=df['rsi'], mode='lines', name='rsi', line=dict(color='grey'))
# rsi_overbought = go.Scatter(x=df.index, y=[60]*len(df), mode='lines', name='rsi_overbought', line=dict(color='blue'))
# rsi_oversold = go.Scatter(x=df.index, y=[50]*len(df), mode='lines', name='rsi_oversold', line=dict(color='blue'))


# fig.add_trace(rsi, row=2, col=1)
# fig.add_trace(rsi_overbought, row=2, col=1)
# fig.add_trace(rsi_oversold, row=2, col=1)

# fig.add_scatter(x=df.index, y=df['rsi'], mode='lines', name='rsi', line=dict(color="orange"))
# fig.add_scatter(x=df.index, y=df['pointpos'], mode="markers",
#                 marker=dict(size=5, color="MediumPurple", symbol='circle'),
#                 name="pivot")

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()